In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('amazon_cleaned_final.csv',nrows=20000)

In [3]:
df

,product_title,average_rating,rating_number,image_url,overall
0,1.2m Brand new Replacement Cable for Sennheise...,3.5,11,https://m.media-amazon.com/images/I/41tniwwRuY...,"Sennheiser ElectronicsHeadphones, Earbuds & Ac..."
1,KKstorm School Headset for School: Student Hea...,3.5,35,https://m.media-amazon.com/images/I/31Yd6YaIVj...,KKstormQUALITY: Premium Quality Wired Headphon...
2,Microsoft Surface Headphones (Renewed),4.2,40,https://m.media-amazon.com/images/I/313ZjJCCCq...,"Amazon RenewedHear crisp, clear audio. Omnison..."
3,"erjigo Earphone, Customized Comfort, GK-GS10Ea...",5.0,5,https://m.media-amazon.com/images/I/41WqjKsKZV...,erjigoQuality assurance and fashionable appear...
4,Youkamoo 4 Pin XLR Balanced Male Compatible fo...,4.5,140,https://m.media-amazon.com/images/I/41abb4Xicx...,Youkamoo4 Pin XLR MaleDual 3.5mm Male8 Core Br...
...,...,...,...,...,...
19995,Hermitshell Hard Travel Case Fits Premium Blue...,4.4,73,https://m.media-amazon.com/images/I/41tRRi48uW...,HermitshellHermitshell Hard Travel Storage Car...
19996,S10+ Ear Buds in Ear Headphones 3.5mm in-Ear E...,3.4,114,https://m.media-amazon.com/images/I/31WJgEnQvm...,snzvok Nope
19997,Genuine Sennheiser Replacement Ear Pads Cushio...,4.5,118,https://th.bing.com/th?id=OIP.nRA6M0oRKNauQsYQ...,SennheiserBrand New 2 Ear pads (one pair) genu...
19998,"BGFOX Earbuds with Microphone, 3.5mm in-Ear Wi...",3.8,197,https://m.media-amazon.com/images/I/31HtJHcLOH...,BGFOXGood Quality Earbuds: The powerful 13.6mm...


In [4]:
df.isna().sum()

product_title     0
average_rating    0
rating_number     0
image_url         0
overall           0
dtype: int64

In [5]:
detail=df['overall']

In [6]:
detail

0        Sennheiser ElectronicsHeadphones, Earbuds & Ac...
1        KKstormQUALITY: Premium Quality Wired Headphon...
2        Amazon RenewedHear crisp, clear audio. Omnison...
3        erjigoQuality assurance and fashionable appear...
4        Youkamoo4 Pin XLR MaleDual 3.5mm Male8 Core Br...
                               ...                        
19995    HermitshellHermitshell Hard Travel Storage Car...
19996                                         snzvok  Nope
19997    SennheiserBrand New 2 Ear pads (one pair) genu...
19998    BGFOXGood Quality Earbuds: The powerful 13.6mm...
19999    WONNIE♥ 【HD Decoding Chip】Ultra-thin 10.5 inch...
Name: overall, Length: 20000, dtype: object

In [7]:
import nltk
# nltk.download('plunkt')
from nltk.tokenize import word_tokenize
detail=detail.apply(lambda x: word_tokenize(x))


In [8]:
detail=detail.apply(lambda x:' '.join(x))

In [9]:
detail=detail.str.replace('[^a-zA-Z0-9+-/]+',' ')

In [10]:
detail=detail.apply(lambda x:' '.join([i for i in word_tokenize(x) if len(i)>2]))

In [11]:
from nltk.stem import PorterStemmer
p=PorterStemmer()
detail=detail.apply(lambda x:[p.stem(i) for i in word_tokenize(x)]).apply( lambda x:' '.join(x))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(stop_words='english')
data=vec.fit_transform(detail)

In [13]:
data

<20000x78741 sparse matrix of type '<class 'numpy.float64'>'
	with 1496413 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(data)

In [15]:
similarity

array([[1.        , 0.00690009, 0.01263814, ..., 0.11842232, 0.02651664,
        0.        ],
       [0.00690009, 1.        , 0.05693743, ..., 0.05841648, 0.13601278,
        0.07152899],
       [0.01263814, 0.05693743, 1.        , ..., 0.01039075, 0.1234697 ,
        0.01769151],
       ...,
       [0.11842232, 0.05841648, 0.01039075, ..., 1.        , 0.02618523,
        0.03146707],
       [0.02651664, 0.13601278, 0.1234697 , ..., 0.02618523, 1.        ,
        0.04148329],
       [0.        , 0.07152899, 0.01769151, ..., 0.03146707, 0.04148329,
        1.        ]])

In [16]:
def recomnendation(product):
    product_index=df.index[df['product_title']==product][0]
    similar_product=list(enumerate(similarity[product_index]))
    similar_product.sort(key=lambda x:x[-1],reverse=True)
    top_similar=similar_product[1:6]
    result=[df.iloc[i[0],0] for i in top_similar]
    img_link=[df.iloc[i[0], 3] for i in top_similar]
    
    return  result,img_link

In [17]:
res,img=recomnendation('Microsoft Surface\xa0Headphones (Renewed)')


In [18]:
for i in res,img:
    print(i)

['Microsoft Surface\xa0Headphones', 'NEW Microsoft Surface Headphones 2 - Light Gray', 'Elesound NC100 Wired Active Noise Cancelling Headphones,Hi-Fi Deep Bass Over Ear Headphones with Microphone,Comfortable Protein Earpads,20H Playtime for Adults Airplane iPhone Android Travel (Black)', 'E7 Active Noise Cancelling Headphones Bluetooth Headphones Over Ear Wireless Headphones with Microphone Deep Bass, Comfortable Protein Earpads, 30HPlaytime for Travel/Work', 'Active Noise Cancelling Headphones,Wireless Bluetooth Headphones Built-in Mic 40 Hours Playtime Wireless Noise Cancelling Headphone 3D Low Bass Tone Fast Charge for Cellphone/Work/Gym/TravelComputers']
['https://m.media-amazon.com/images/I/313ZjJCCCqL._AC_.jpg', 'https://m.media-amazon.com/images/I/21T0PWJzRRL._AC_.jpg', 'https://m.media-amazon.com/images/I/41wO2OH4CtL._AC_.jpg', 'https://m.media-amazon.com/images/I/41BuFk77+8L._AC_.jpg', 'https://m.media-amazon.com/images/I/31HQVFSfNGL._AC_.jpg']


In [19]:
df['image_url'][0]

'https://m.media-amazon.com/images/I/41tniwwRuYL._AC_.jpg'

In [20]:
import pickle
pickle.dump(df,open('product.pkl','wb'))

In [21]:
pickle.dump(similarity,open('product_similarity.pkl','wb'))